# TASK 4: Predictive Modelling
**Build and evaluate predictive models that form the core of a dynamic, risk-based pricing system**

In [ ]:
import sys
import os
import pandas as pd
from glob import glob

In [1]:
from pathlib import Path
from importlib import reload
# add the project root to the path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

NameError: name 'sys' is not defined